# 事故情報（accidentインデックス）の事故内容に類似する事故を検索(anzenインデックス)し、その状況や原因、対策をサジェストする。

## 1. まず、事故情報（accidentインデックス）から類似検索したい事故を探す。
* ここでは単純なキーワード検索で事故情報を検索する。

In [180]:
import elasticsearch, json
import pandas as pd

HOST = "elasticsearch"
PORT = "9200"
INDEX_NAME = "accident"
SCROLL_TIME = '1m'
SCROLL_SIZE = 10
pd.set_option("display.max_colwidth", 100)

search_word = "カバー"

client = elasticsearch.Elasticsearch(HOST + ":" + PORT)

results = []
scroll_results = client.search(
    index = INDEX_NAME,
    scroll = SCROLL_TIME,
    size = SCROLL_SIZE,
    body = {"query": { "match": {"sentence":search_word} }}
)

sid = scroll_results['_scroll_id']
scroll_results = scroll_results['hits']['hits']
scroll_size = len(scroll_results)

for scroll_result in scroll_results:
    results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['category'], "sentence":scroll_result['_source']['sentence']})

while scroll_size > 0:
    scroll_results = client.scroll(
        scroll_id = sid,
        scroll = SCROLL_TIME )
    sid = scroll_results['_scroll_id']
    scroll_results = scroll_results['hits']['hits']
    scroll_size = len(scroll_results)
    
    for scroll_result in scroll_results:
        results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['category'], "sentence":scroll_result['_source']['sentence']})

df = pd.DataFrame(results)

# ここでは最初の10件のみ表示させておく
df = df.head(10)
df_style = df.style.set_properties(subset=["sentence"], **{'text-align': 'left'})
df_style

,id,category,sentence
0,BpP_N3ABaO95s7ZFY6wW,製造業,レールを切断する鋸断機のカバーを開けて鋸断機とカバーの間に立っていたところ、鋸断機横の台車（自動運転）が動き出してカバーを押し、鋸断機とカバーの間に全身をはさまれた状態で発見されたもの。
1,fJMBOHABaO95s7ZFKdCM,建設業,杭打ち機のメンテナンス作業中、中間振れ止めカバーが落下し、下部振れ止めカバーとの間に挟まれた。
2,J5P9N3ABaO95s7ZFZIOn,建設業,めっき槽に付属する配管の塗装作業中、めっき槽上に取り付けてあったカバーが外されていたが、被災者はカバーがあると勘違いしてカバーの上を渡ろうとして、めっき槽の中に落ちて被災した。
3,nZP_N3ABaO95s7ZFs7KV,製造業,フェルト製品の製造作業中において、綿を送り出すための装置の下部カバー内で綿が詰まったため、被災者が詰まった綿をカバー下部から取り出していた所、綿が一度に大量に落ち、カバーから引き抜いた際にカバー下部にある脱水機のシュート部で回転していた綿に引き込まれ、脱水機の壁面で強打した。
4,L5P8N3ABaO95s7ZFlHK_,建設業,天井クレーン用走行レールの点検作業後、梯子で地上に降りる途中、両手に持っていた車輪カバーのカバーが開いたため体勢が崩れ、約６．３ｍ下の床に転落した。
5,IZMAOHABaO95s7ZFhMNR,建設業,アース・オーガーで穴明け作業をするためドリルパイプの位置調整をしていたときに、土砂飛散防止用カバーを吊っていたワイヤーロープが切断したためカバーが落下して当った。
6,V5MBOHABaO95s7ZFo9px,製造業,構内の資材置場で鉄製機械カバー（重量約９００kg）をトラックから降ろす作業中、荷台にあった同カバーが突風にあおられて倒壊し、その場を歩行していて下敷きとなった。
7,SpMBOHABaO95s7ZFJtDh,製造業,変圧器用タンクカバーの溶接作業に際し、設置位置の台を修正するため、カバー（重さ3.5トン）を天井クレーンでつり上げたところ、ワイヤロープ（径9.3mm）が破断し、当該カバーが倒れ、その下敷きとなった。
8,EJP7N3ABaO95s7ZFoF73,製造業,発電設備であるガスタービンの鉄製防音カバー（天井部分と側壁2面がボルトにより結合されたもので、質量約1.1t）内側のシート張りと解体作業中、倒壊防止のための補強が不十分であったためカバー倒壊し、中で作業していた2名のうち1名がカバーの下敷きになり死亡し、１名が骨盤、脚を骨折した。
9,k5P8N3ABaO95s7ZFwHUQ,建設業,自転車用道路の舗装工事中、タイヤローラーを後進させていたところ、タイヤローラー座席の台座カバー取付部が腐食していたためカバーと座席が外れ、座席ごと車外へ転落した際タイヤローラーにひかれた。


## 2. 上記の検索結果の中から、類似検索したい事故を指定する。
* キーワード検索した結果の中から、類似検索したい事故を探す。
* 類似検索したい事故の一番左の列の番号を、target_idに設定する。
* 選択した事故の内容が表示される。

In [181]:
# 特定の行を参照する。0始まり
target_id = 0
target = df.loc[target_id]
print("以下の事故について、類似事象を抽出します。")
target

以下の事故について、類似事象を抽出します。


id                                                                                   BpP_N3ABaO95s7ZFY6wW
category                                                                                              製造業
sentence    レールを切断する鋸断機のカバーを開けて鋸断機とカバーの間に立っていたところ、鋸断機横の台車（自動運転）が動き出してカバーを押し、鋸断機とカバーの間に全身をはさまれた状態で発見されたもの。
Name: 0, dtype: object

## 3. 上記で指定した事故に類似する事故・状況・原因・対策を検索し、トップ10を表示する

### 3-1. 指定した事故の文章ベクトルを取得する

In [182]:
# 対象の文章ベクトルを取得する。
id = target['id']
target_detail = client.search(
    index = "accident",
    body = {"query": { "match": {"_id":id} }}
)
target_vector = target_detail['hits']['hits'][0]['_source']['scdv_vector']

### 3-2. 類似検索のためのクエリ
* 事故の内容に関する類似検索なので、タイトルと状況のみを検索対象としている。

In [183]:
# 類似検索用のクエリ
# 事故内容に対して、タイトルと事象が類似する事例を抽出する。
script_query = {
    "script_score": {
        "query": {
            "bool" : {
                "should" : [ { 
                    "bool" : {
                        "must": [
                            {"term": { "category":"title"}},
                            {"exists":{"field": "vector"}}
                        ]
                    } 
                }, {
                    "bool": {
                        "must" : [
                            {"term": { "category":"situation"}},
                            {"exists":{"field": "vector"}}
                        ]
                    }
                }  ]
            }
        },
        "script": {
            "source": "cosineSimilarity(params.query_vector, doc['vector']) + 1.0",
            "params": {"query_vector": target_vector}
        }
    }
}

### 3-3. 類似検索の実行
* 指定した事故に類似する、タイトルと状況のトップ10が抽出される。

In [184]:
# 類似する文書を検索する。ここでは、類似するタイトルと状況のみとなる。
responses = client.search(
    index="anzen",
    body={
        "size": SCROLL_SIZE,
        "query": script_query,
        "_source": {"includes": ["_id", "doc_id", "category", "text_id", "text"]}
    }
)

# 結果の整形処理
responses = responses['hits']['hits']
similar = []
for response in responses:
    id = response['_id']
    score = response['_score']
    doc_id = response['_source']['doc_id']
    text_id = response['_source']['text_id']
    category = response['_source']['category']
    text = response['_source']['text']
    similar.append({'SCORE':score,'doc_id':doc_id,'text_id':text_id, "category":category, "text":text})

sim_df = pd.DataFrame(similar)
sim_df_style = sim_df.style.set_properties(subset=["text"], **{'text-align': 'left'})
sim_df_style

,SCORE,doc_id,text_id,category,text
0,1.68851,100858,100858_s_06,situation,手順は、まず新しいメーター取り付け用の配管を準備し、続いて穴の中に入って既設の配管(25mmの鋳鉄管)の両側を、ガスの漏れ止めテープを切断面に巻きながら金切鋸で切断し、切断が終了したところで、用意した新しい配管のカップリング切断面に突っ込みナットで締め付けるというものであった。
1,1.65278,101061,101061_s_06,situation,Aは、操作盤が遠いため足元のマットスイッチを踏み成型機を停止させた後、成型機のドラム上に身を乗り出したところ、バランスを崩し、このためマットスイッチから足が離れてしまい、成型機が突然作動して、タイヤを押さえるガイドが降下し、ドラムとの間にはさまれて被災した。
2,1.65261,101210,101210_s_02,situation,災害発生当日、作業者Aは、ホットプレスおよびダブルソーを使用して合板製造作業に従事していた。ホットプレスを使用した圧着作業は、午前中に予定数を終わったため、午後はダブルソーを使用して、圧着した合板を規定の寸法に切断する作業を行っていた。午後の作業を始めて約1時間が経過したとき、ダブルソーの合板を送給するローラーに合板を切断した端材やゴミが詰まり、合板を規程寸法どおりに切断することができなくなった。そこで、Aは、ダブルソーを動かしたまま、ダブルソーの側面からローラーに詰まった端材やゴミを取り除いていたところ、誤ってローラーに触れ、回転中のローラーに巻き込まれた。Aは直ちに救出され、病院に搬送されたが、間もなく死亡した。
3,1.65108,100581,100581_s_04,situation,災害発生当日、敷地内で、このエアバッグを電流により作動させてバッグを膨脹させたのちに廃棄する作業を開始した。Aはホイールを外した古タイヤの内側にエアバッグを自動車用電線を用いて固定し、その上にさらに3本の古タイヤを重ねた。この時、下から2本目と4本目のタイヤにはホイールが付いたままであった。次にエアバッグの作動用電気導線を1mほど離れたところにおいたバッテリーのところにいるBに渡し、重ねられたタイヤを上から体重を掛けて押さえ付けて、Bにバッテリーによる通電を命じた。Aの合図によりBがこの電気導線をバッテリーの端子に接続して、タイヤに背を向けて通電したところ、「ドスン」という音を聞いたので振り向くと、タイヤとともに宙に浮いたA が前頭部からアスファルト地面に激突するところであった。
4,1.64811,621,621_s_05,situation,1本目の鉄筋を切断し、2本目の切断にかかったとき、被災者は感電し、ディスクグラインダをスイッチの入ったまま投げ出した。
5,1.64518,100848,100848_s_06,situation,運転者Cはブル・ドーザーで補助者Dとともに現場に到着し、向きを180度転回し、エンジンを停止してから運転席を降りた。Dは、後部ウインチの牽引用ワイヤロープの先端近くが緩み、ストランドが一本飛び出しているのをみつけ、Cがワイヤーをつかみ、Dがカッターで切断しようとしたがうまくいかなかった。Cは、ウインチを作動させてワイヤロープを引き出し地上に下ろして切断することにし、運転席の外から手を延ばしてエンジンをかけたところ、突然ブル・ドーザーが後進し、Dがひかれ、救出しようとしたCも左腕をクローラにひかれた。
6,1.63589,101223,101223_s_04,situation,災害発生当日、被災者は単独で丸のこ盤を使用して切断作業を行っていた。作業中、切断された合板を送り出すローラーにゴミの付着が認められたので、調べたところ、ローラーとフレームとの間に端材が詰まっているのが確認された。そこで、端材を取り除こうとして手を伸ばしたところ、ローラーに巻き込まれ被災したものである。
7,1.63364,545,545_s_03,situation,Z工業では、このマーキング切断に半自動ガス切断機を使用しているが、この半自動ガス切断機は、溶断しようとする鋼板の上に切断機を置き、火口に点火してスイッチを入れると自動的に溶断が開始されるものである。切断機の下部に車輪が付いているため、切断機が自動的に移動して順次溶断していくようになっているが、移動する方向は常に手でガイドしてやる必要があるため、完全自動でなく半自動となっている。
8,1.62858,100107,100107_s_02,situation,災害発生当日の始業時、土捨て場に放置されていたブルドーザを引き出し、修理可能か否かを確認するよう指示されたクレーン運転手は、玉掛け作業者と共にジブ伸縮式のホイールクレーン(つり上げ荷重26トン)で現場に向かい、まず、ブルドーザ周辺の土砂をドラグショベルを用いて取り除いた。午前9時頃ワイヤロープをブルドーザ後部に掛けて、引き出そうとロープを巻き上げたが、ホイールクレーンの過負荷防止装置の自動停止機能が作動し、巻き上げが停止した。その時ブルドーザは全く移動せず、元の位置のままであり、浮き上がることもなかった。そこで、クレーン運転手は過負荷防止装置の自動停止機能が作動しなければブルドーザを引き出せると考え、過負荷防止装置の自動停止スイッチを「解除」に設定し、再び巻き上げを開始したところ、ホイールクレーンの3段目のジブ及び起伏シリンダーが「へ」の字形に変形し、折損した。
9,1.62307,101117,101117_s_10,situation,グラインダーの回転数と切断といしの最高使用周速度は適切であり、といし覆いも研削盤構造規格に適合していた。切断面にといしがはさまった際、といしの側面に負荷が加わり、といしに小さな亀裂が生じていたため、無負荷状態での回転では破裂しなかったものの、パイプに接触した衝撃でといしが破裂したものである。この日、Ａは災害発生時までに、摩耗により３枚のといしの取替えを行っていたほか、切断中にといしが切断面にはさまるトラブルが７～８回発生していた。


### 3-4. 類似事例の検索
* 上記で抽出したタイトル、状況について、該当する事例を取得し表示する。

In [188]:
# 指定したdoc_idの事例（タイトル、状況、原因、対策）を検索する。
def search_doc(doc_id):
    SCROLL_TIME = '1m'
    SCROLL_SIZE = 10
    results = []
    scroll_results = client.search(
        index = "anzen",
        scroll = SCROLL_TIME,
        size = SCROLL_SIZE,
        body = {
            "query": { "term": {"doc_id":doc_id} },
            "_source": {"includes": ["doc_id", "category", "text_id", "text"]}
        }
    )

    sid = scroll_results['_scroll_id']
    scroll_results = scroll_results['hits']['hits']
    scroll_size = len(scroll_results)

    for scroll_result in scroll_results:
        results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['category'], "text_id":scroll_result["_source"]["text_id"],"text":scroll_result['_source']['text']})

    while scroll_size > 0:
        scroll_results = client.scroll(
            scroll_id = sid,
            scroll = SCROLL_TIME )
        sid = scroll_results['_scroll_id']
        scroll_results = scroll_results['hits']['hits']
        scroll_size = len(scroll_results)

        for scroll_result in scroll_results:
            results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['category'], "text_id":scroll_result["_source"]["text_id"],"text":scroll_result['_source']['text']})
    return results

In [189]:
# 類似する事例を検索し、結果を整形して出力する。
results = []

for index, row in sim_df.iterrows():
    doc_id = row.doc_id
    
    docs = search_doc(doc_id)
    
    docs_sorted = sorted(docs, key=lambda x:x['text_id'])
    title = ([result['text'] for result in docs_sorted  if result['category'] == 'title'])
    situations = ([result['text'] for result in docs_sorted  if result['category'] == 'situation'])
    situations = "\n".join(situations)    
    causes = ([result['text'] for result in docs_sorted  if result['category'] == 'cause'])
    causes = "\n".join(causes)
    measures = ([result['text'] for result in docs_sorted  if result['category'] == 'measures'])
    measures = "\n".join(measures)
    line = {"件名":title[0], "状況:":situations, "原因":causes, "対策":measures}
    results.append(line)

df = pd.DataFrame(results)
df_style = df.style.set_properties(subset=["件名", "状況", "原因", "対策"], **{'text-align': 'left'})
df_style

,件名,状況:,原因,対策
0,プロパンガスメーターの取替え作業中、酸素欠乏症になる,この災害は、個人住宅のガスメーター取替え作業中に発生したものである。 災害は集合プロパンガス供給方式となっている団地の個人住宅の敷地内で発生したが、この住宅のガスメーターは住宅の奥の方に取り付けられているため検針のときに不便であったことから、住宅の持ち主から移設の申し出があり、被災者が1月ほど前に下見を行って移設場所および移設方法を決定した。 災害発生当日の午前11時30分頃から既設の埋設配管がある場所をスコップで深さ1m、広さ0.8m×0.7mの穴を掘り、既設配管を露出させた後、被災者はいったん現場を離れ、午後2時30分頃に戻ってきて移設の作業を再開した。 手順は、まず新しいメーター取り付け用の配管を準備し、続いて穴の中に入って既設の配管(25mmの鋳鉄管)の両側を、ガスの漏れ止めテープを切断面に巻きながら金切鋸で切断し、切断が終了したところで、用意した新しい配管のカップリング切断面に突っ込みナットで締め付けるというものであった。 午後3時頃、住宅の持ち主が作業の様子を見に行ったところ、ガスの臭いがして、被災者が穴の中で掘削面にもたれ掛るようにしてしゃがみ込み、眼を閉じて身体を小刻みに震わせていた。直ちに消防署に連絡し病院に移送したが、翌日に無酸素脳症のため死亡した。,この原因としては、次のようなことが考えられる。 1 ガスの供給を止めずに作業を行ったこと このガスメーター移設作業は、集合供給方式を採っている団地内であったため、他の家庭への供給停止を避けるため、また、供給停止に伴う他の家庭への開始時および復旧時の連絡、確認の煩雑さを避けるため、ガスを供給したまま行われた。 しかし、狭い掘削穴の中での作業であったため作業に手間取り、また、切断した配管からのガスをテープでは完全には封じることができなかったため、狭い掘削内にプロパンガスが充満し、酸素欠乏状態に至ったものと推定される。 2 呼吸用保護具を使用しなかったこと 作業を行っていた場所の周囲には物置、塀があって通風が不十分な場所であったのに、強制換気あるいはホースマスクの使用を行っていなかった。 なお、会社にはホースマスクの設備が１セット用意されていたが、同種工事で使用された実績はなく、また、会社から使用の指示もしていなかった。 3 酸素欠乏についての教育等を実施していなかったこと 被災者は、会社の保安工事課長として作業者を指揮監督する立場にあったが、酸素欠乏に関する知識は無く、また、会社として酸素欠乏危険とその予防に関する教育を実施していなかった。 なお、会社には、酸素濃度測定器も備え付けていなかった。,"同種災害の防止のためには、次のような対策の徹底が必要である。 1 安全な作業手順を定めること プロパンガスは、気体比重が1.6であることから、穴の中等に漏洩した場合には低いところに滞留し、高濃度になると酸素欠乏症になるので、ガスの供給を行ったまま作業を行うことは避け、ガスの供給を遮断する作業手順を定め、関係作業者に徹底する。(酸欠則第23条の2関連) また、ガスの一時遮断による作業を原則とし、周囲の住宅に対する連絡、確認要領についても明確に定め徹底する。 2 酸素欠乏危険に関する教育を実施すること プロパンガスは、爆発危険のほか酸素欠乏危険があるので、メーターの取り付け・移設作業等に従事する作業員に対して、あらかじめ爆発危険および酸素欠乏危険とその防止策について十分な教育を実施する。(酸欠則第12条関連) 3 空気呼吸器等の準備と使用を徹底すること 酸素欠乏危険等のある作業については、空気中の酸素濃度が18%以上になるように強制換気するとともに、必要に応じホースマスク等の空気呼吸器を使用させる。 そのため、酸素濃度測定器、強制換気装置、空気呼吸器等を必要数準備し、保守管理を行うとともに、関係作業者に使用方法を教育する。(酸欠則第3,4,5,7条,5条の2関連) 4 安全衛生管理を十分に行うこと 事業者は、受注した作業に関する危険有害性をあらかじめ検討する体制を整備し、作業手順書の作成、安全衛生教育の実施、必要な機器材の整備等を行う。 また、作業開始前のKYK(危険予知活動)の導入などについて検討するとともに、経営トップは定期あるいは随時に作業現場を巡視し、必要な指示を行う。"
1,タイヤの成型作業中、成型機ドラムとガイドとにはさまれ死亡,この災害はトラック・バス用ラジアルタイヤ製造工程において、タイヤの成型作業中に発生したものである。 ラジアルタイヤの成型作業は、タイヤのチューブにあたるインナーライナーを成型機のドラムに貼り、その後、他の部材を順次貼っていくものである。 作業者Aは、タイヤ成型作業中、最初に成型機のドラムに貼り付けたインナーライナーに不良部分を見つけたので、不良インナーライナーを成型機から取り出そうとした。 Aは、操作盤が遠いため足元のマットスイッチを踏み成型機を停止させた後、成型機のドラム上に身を乗り出したところ、バランスを崩し、このためマットスイッチから足が離れてしまい、成型機が突然作動して、タイヤを押さえるガイドが降下し、ドラムとの間にはさまれて被災した。 タイヤ成型機には身体が危険範囲に入った場合に成型機を自動停止させる安全装置が設置されておらず、成型機の操作盤も作業位置から離れていた。 この工場では、非定常作業の作業手順書を作成しておらず、作業者への安全衛生教育も実施していなかった。,この災害の原因としては、次のことが考えられる。 1 身体が成型機にはさまれる危険性のあるインナーライナーの取出し等の作業について、自動停止させる安全装置が設置されていなかったこと 2 作業位置から操作できる位置に成型機の操作盤を設置していなかったこと 3 非定常作業の作業手順書（非定常作業では操作盤で機械を手動に切り替えて作業を行う）を作成していなかったこと 4 作業者への安全衛生教育を実施していなかったこと,同種災害を防止するためには、次のような対策が必要である。 1 インナーライナ－の取出し等の作業では、成型機を自動停止させる安全装置を設置すること 2 作業位置から操作できる位置に成型機の操作盤を移動し、作業者がいつでも成型機を停止できるようにすること 3 非定常作業の作業手順書を作成して、作業者に徹底させること 4 作業の実態を十分に把握した上で、作業の近道・省略行為等によるヒューマンエラーを防止するための安全衛生教育を実施すること
2,合板切断用ダブルソーの合板を搬送するローラーに巻き込まれて死亡,この災害は、合板製造工場で合板切断用ダブルソーの合板を送給するローラー(長さ2m、直径15cmの円柱状、金属製)に作業者が巻き込まれて死亡したものである。 災害発生当日、作業者Aは、ホットプレスおよびダブルソーを使用して合板製造作業に従事していた。ホットプレスを使用した圧着作業は、午前中に予定数を終わったため、午後はダブルソーを使用して、圧着した合板を規定の寸法に切断する作業を行っていた。午後の作業を始めて約1時間が経過したとき、ダブルソーの合板を送給するローラーに合板を切断した端材やゴミが詰まり、合板を規程寸法どおりに切断することができなくなった。そこで、Aは、ダブルソーを動かしたまま、ダブルソーの側面からローラーに詰まった端材やゴミを取り除いていたところ、誤ってローラーに触れ、回転中のローラーに巻き込まれた。Aは直ちに救出され、病院に搬送されたが、間もなく死亡した。 ダブルソーの側面には作業者が回転中のローラーに接触することを防止するためのガード等は設けられておらず、緊急時に直ちに停止させる非常停止スイッチも設けられていなかった。 Aがはさまれたダブルソーのローラーは、端材やゴミの詰まりが1日に数回程度発生しており、その都度、除去作業を行っていた。端材やゴミの除去作業はダブルソーを止めて行うことが作業手順書に記載されていたが、Aは、止めずに除去作業を行っていた。 また、端材やゴミの除去作業を行うときは、専用の手工具を使用することになっているが、この手工具はダブルソーから離れた保管場所に置かれており、Aは普段から手工具を使用せず、素手で行っていた。,この災害の原因としては、次のようなことが考えられる。 1 回転中のローラーに作業者が容易に触れることができる危険な状態であったこと 2 ローラーに詰まった端材やゴミを除去する際に、ダブルソーを停止しなかったこと 3 ローラーに詰まった端材やゴミを除去する際、専用の手工具を使用せず、素手で行ったこと,同種災害の防止のためには、次のような対策の徹底が必要である。 1 作業者が接触し巻き込まれるおそれのあるローラーにはガード等を設けること ダブルソー等の機械の使用段階においては、リスクアセスメントを行い、作業者が接触し巻き込まれるおそれのある個所には、作業者が可動部分に接触すること防止するためのガード等を設ける。また、ガード等を開けたときは直ちに機械が停止し、閉めた後でなければ再起動しない構造のものとすることが大切である。 さらに、万一、ローラーに巻き込まれそうになっても、直ちに停止させることができる非常停止スイッチを作業位置から離れずに操作できる場所に設置することも必要である。 2 機械の点検・清掃を行う際は、当該機械を必ず停止させて作業を行うことを作業者に徹底すること 機械の点検・清掃は、当該機械を必ず停止させて作業を行うことを作業手順書に盛り込むととも